# Side effects Data

To run this you first have to run preprocessing/01_classifying text lines and also preprocessing/02_ms-classification and preprocessing 04/llama-finetune. This ensures that you have the same data.

In [1]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import pandas as pd

from datasets import Dataset

import json

from datasets import DatasetDict

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"text-finetune/kisim_diagnoses_combined.csv")

In [3]:
keywords = ["Nebenwirkung", " NW "]

# Filter out all the entries that contain more
df_se = df[df["text"].str.contains("|".join(keywords), case=False, na=False)]

# Filter only one text per rid (so no doublte texts)
df_se = df_se.drop_duplicates(subset="research_id")

In [4]:
# Print the lines that contain the keyword
for text in df_se["text"]:
    text = text.splitlines()
    for line in text:
        if any(keyword in line for keyword in keywords):
            print(line)
            print("-" * 50)

INDENT 12/2014 - 08/2015 Tecfidera, sistiert bei gastrointestinalen Nebenwirkungen
--------------------------------------------------
INDENT 11/97 - 02/05: Rebif (Stopp wegen Nebenwirkungen)
--------------------------------------------------
INDENT 08/08 - 10/09: Rebif 3x44 ug (erneuter Beginn; wg. Nebenwirkungen abgesetzt) 
--------------------------------------------------
Nicht näher bezeichnete unerwünschte Nebenwirkung eines Arzneimittels oder einer Droge
--------------------------------------------------
INDENT 11/2014 - 08/2017 Tecfidera, keine relevanten Nebenwirkungen
--------------------------------------------------
INDENT St.n. Therapie mit Tecfidera 02/2019 (Abbruch bei Gastrointestinalen Nebenwirkungen)
--------------------------------------------------
INDENT St.n. symptomatischer Therapie mit Fampyra (insgesamt 2 Therapieversuche, gestoppt bei Wirkungslosogkeit und i.R. der kardialen Vorbefunden), Therapieversuch mit Modasomil/Remeron (keine Besserung), Venlafaxin (bei 

In [5]:
len(df_se)

131

Sample 100 of them

In [6]:
# Sample 100 entries
df_se_sample = df_se.sample(100, random_state=42)

In [7]:

for text in df_se_sample["text"]:
    text = text.splitlines()
    for idx, line in enumerate(text):
        if any(keyword in line for keyword in keywords):
            print(text[idx:idx+3])
            print("=" * 50)


['INDENT 10/2019: Gabapentin bis 100mg/d, Phenytoinaufsättigung 09.10. - 13.10.2019 (gestoppt bei Intoxikation mit Nebenwirkungen)', 'INDENT 01/2020: Gabapentin bis 300mg/d, Phenytoin p.o. 200mg/d bis OP 02/2020, dann Gabapentin stopp in 05/2020. ', 'INDENT 02/2020: erneute perkutane Thermokoagulation Ganglion gasseri']
['INDENT Lymphopenie bekannte NW unter Avonex', 'INDENT Stopp Avonex 26.08.14, da Fumaderm als Therapie für PsO und MS eingeleitet wurde, aber überlappend mit Fumaderm (Beginn 31.07.2014)']
['INDENT 10/05-10/13: Betaferon, darunter kutane Verhärtungen und grippale Nebenwirkungen', 'INDENT seit 07.10.13: Gilenya']
['INDENT Nebenwirkungen: Diabetes mellitus (Prednison), Osteoporose (Prednison), Neutropenie (Imurek), Niereninsuffizienz (multifaktoriell, zum Teil unter Ciclosporin)', 'INDENT Abstossungshistorie', 'INDENT ISHT 3 A (therapiebedürftige Abstossungsreaktion) am 17.07.18 (15. EMB) und 30.01.18 (7. EMB)']
['INDENT - 2005 für drei Monate Avonex, wegen starker gripp

In [8]:
# Create HF dataset
os.makedirs(paths.DATA_PATH_PREPROCESSED/"side-effects", exist_ok=True)
df_se_sample.rename(columns={"research_id": "rid"}, inplace=True)
df_se_sample = df_se_sample[["rid", "text"]]
df_se_sample.to_csv(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample.csv", index=False)
ds_se = Dataset.from_pandas(df_se_sample, preserve_index=False)

In [9]:
# Save the dataset
ds_se.save_to_disk(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

# Prompting

In [10]:
# Prompt
task_instruction ="""Your task is to extract medications that showed adverse side effects from a medical report. 
The input for this task is a medical report in text form, and the output should be a complete list of dictionaries (one per medication) with the following keys:

- "medication" (str): the name of the medication
- "side_effect" (str): the side effect of the medication

The output format should look like this:
[
    {"medication": "MedicationName1", "side_effect": "SideEffect1"},
    {"medication": "MedicationName2", "side_effect": "SideEffect2"},
    ...
]

- The "MedicationName" can consist of multiple words with whitespace and should be returned as a single string. If you don't find the medication that cause the side-effect, the output should be "unknown".
- The "SideEffect" describes the specific adverse event that was caused by the medication. Sometimes it is not explicitly mentioned; in this case, it should be represented as "unknown".
- If you can't find any mentions in the input of medications that showed adverse side effects, the output should be [{"medication": "unknown", "side_effect": "unknown"},]

Only medications that showed adverse side effects should be extracted. If a medication is mentioned but no side effect is mentioned, it should not be included in the output.
The text is in German, watch for the keywords "Nebenwirkung", "Begleiterscheinung", "Begleitsymptom", "Begleitphänomen" to identify the side effects.

"""

system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don’t know the answer to a question, please don’t share false information.
"""

with open(paths.DATA_PATH_PREPROCESSED/"side-effects/task_instruction.txt", "w") as file:
    file.write(task_instruction)
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/system_prompt.txt", "w") as file:
    file.write(system_prompt)

In [11]:
examples = [
    {"text": "INDENT 12/2014 - 08/2015: Behandlung mit Tecfidera unterbrochen aufgrund von gastrointestinalen Nebenwirkungen", "labels" : '[{"medication": "Tecfidera", "side_effect": "gastrointestinalen Nebenwirkungen"}]'},
    {"text": "11/97 - 02/05: Einnahme von Rebif beendet aufgrund von Nebenwirkungen", "labels" : '[{"medication": "Rebif", "side_effect": "unknown"}]'},
    {"text": "Unspezifische unerwünschte Reaktion auf ein Medikament oder eine Substanz", "labels" : '[{"medication": "unknown", "side_effect": "unknown"}]'},
    {"text" : "die Diagnose ist eine klinisch Isoliertes Syndrom (CIS)", "labels" : '[{"medication": "unknown", "side_effect": "unknown"}]'},
    {"text": "INDENT St.n. symptomatischer Therapie mit Fampyra (insgesamt 2 Therapieversuche, gestoppt bei Wirkungslosogkeit und i.R. der kardialen Vorbefunden), Therapieversuch mit Modasomil/Remeron (keine Besserung), Venlafaxin (bei Nebenwirkungen gestoppt), Sirdalud", "labels" : '[{"medication": "Venlafaxin", "side_effect": "unknown"}]'},
    {"text": "10/2019: Beginn mit Gabapentin bis zu einer Dosis von 100 mg täglich. Phenytoinaufsättigung vom 09.10. bis 13.10.2019 (wurde aufgrund von Intoxikation mit Nebenwirkungen abgebrochen). 01/2020: Gabapentin bis zu einer Dosis von 300 mg täglich, Phenytoin oral 200 mg täglich bis zur Operation im Februar 2020, dann Absetzen von Gabapentin im Mai 2020. 02/2020: Erneute perkutane Thermokoagulation des Ganglion gasseri.", "labels": '[{"medication": "Phenytoinaufsättigung", "side_effect": "Intoxikation"}]'},
    {"text": "Bekannte Nebenwirkung der Lymphozytopenie unter Medrol. Avonex wurde am 26.08.14 abgesetzt, da Fumaderm zur Behandlung von Psoriasis und Multipler Sklerose begonnen wurde, jedoch mit Überlappung von Fumaderm (Beginn am 31.07.2014).", "labels": '[{"medication": "Medrol", "side_effect": "Lymphozytopenie"}]'},
    {"text": "10/05-10/13: Behandlung mit Betaferon, was zu kutanen Verhärtungen und grippeähnlichen Nebenwirkungen führte. Seit dem 07.10.13 Behandlung mit Gilenya.", "labels": '[{"medication": "Betaferon", "side_effect": "kutanen Verhärtungen und grippeähnliche Nebenwirkungen"}]'},
    {"text": "Im Jahr 2005 für drei Monate Behandlung mit Pantoprazol, jedoch aufgrund starker grippeähnlicher Nebenwirkungen und Spritzenphobie abgebrochen. Seit dem 06.02.2007 Teilnahme an der FREEDOMS-Studie, später an der FREEDOMS Extension.", "labels": '[{"medication": "Pantoprazol", "side_effect": "grippeähnliche Nebenwirkungen"}]'},
    {"text": "Paracetamol 89ug 1/ Woche seit 07/2017; Umstellung aufgrund steigender grippaler Nebenwirkungen', 'INDENT Seit 17.10.2014 Dimethylfumarat (Tecfidera), bisher gut vertragen']", "labels": '[{"medication": "Paracetamol", "side_effect": "grippeähnliche Nebenwirkungen"}]'},
]

# Save dictionary
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/examples.json", "w") as file:
    json.dump(examples, file, indent=4)

# S2A

As the line classifier was trained very unspecific in regards to medication (medms and the rest of medications was part of leftover category). Will train again.
For this will load line_labelled data and put everything medication in class 0, everything else in class 1.

In [12]:
# Load csv
df_med = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_clean.csv")

# Labels are 0 if class == "medms" or "medo", 1 otherwise
df_med["labels"] = df_med["class"].apply(lambda x: 0 if x in ["medms", "medo"] else 1)

# Train val split
df_train, df_val = train_test_split(df_med, test_size=0.2, random_state=42, stratify=df_med["labels"])

# Oversample the minority class

In [13]:
# Create huggingface datasetDict
ds_train = Dataset.from_pandas(df_train, preserve_index=False)
ds_val = Dataset.from_pandas(df_val, preserve_index=False)

ds_dict = DatasetDict({"train": ds_train, "val": ds_val})

# Save the dataset
ds_dict.save_to_disk(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_medication_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1233 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/309 [00:00<?, ? examples/s]